# Control allocation example for MilliAmpere1

## Reference filter based thrust allocation

For the sake of testing, set `PYTHONPATH` so notebook can be run without installing the package.

In [1]:
import sys
sys.path.insert(0, "../src")

First, import the control allocation packge

In [2]:
import skadipy
import skadipy.allocator.reference_filters as skadirf
import ipywidgets as widgets
from IPython.display import display
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button

# Use interactive backend
%matplotlib qt5

Now, we describe the thrusters. There are different options for thrusters. These are `Fixed`, `Aximuth`, `Vectored`.
It is defined by the position of the thruster and the direction that its directed at.
Orientation of the thruster is defined using quaternions.

Depending on the allocator choice, `extra_attributes` dictionary changes.
For milliAmpere1, we use Quadratic Programming allocator. It takes `limits` list to set thrust limits for the thrusters.

X axis is forward, Y axis is starboard, Z axis is down.
For simplicty we will assume that the ship is symmetric and that the thrusters are
symmetrically placed. The azimuthing thrusters are also fixed in the vertical plane.

In [3]:

ma_bow_port = skadipy.actuator.Azimuth(
    position=skadipy.toolbox.Point([0.3, -0.1, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), angle=(3*np.pi / 4.0)),
    extra_attributes={
        "name": "bow_port",
        "rate_limit": 1.0,
        "saturation_limit": 1.0,
        "reference_angle": 0.0
    }
)
ma_bow_starboard = skadipy.actuator.Azimuth(
    position=skadipy.toolbox.Point([0.3, 0.1, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), angle=(-3*np.pi / 4.0)),
    extra_attributes={
        "name": "bow_starboard",
        "rate_limit": 1.0,
        "saturation_limit": 1.0,
        "reference_angle": 0.0
    }
)
ma_aft_port = skadipy.actuator.Azimuth(
    position=skadipy.toolbox.Point([-0.3, -0.1, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), angle=(np.pi / 4.0)),
    extra_attributes={
        "name": "aft_port",
        "rate_limit": 1.0,
        "saturation_limit": 1.0,
        "reference_angle": 0.0
    }
)
ma_aft_starboard = skadipy.actuator.Azimuth(
    position=skadipy.toolbox.Point([-0.3, 0.1, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), angle=(-np.pi / 4.0)),
    extra_attributes={
        "name": "aft_starboard",
        "rate_limit": 1.0,
        "saturation_limit": 1.0,
        "reference_angle": 0.0
    }
)

# Put all actuators in a list and create the allocator object
actuators = [
    ma_bow_port,
    ma_bow_starboard,
    ma_aft_port,
    ma_aft_starboard

]

Now we need to describe what degrees of freedom we want to control.
To specify that, we use `ForceTorqueComponent` class.
Possible options are

- `ForceTorqueCompontent.X` for surge
- `ForceTorqueCompontent.Y` for sway
- `ForceTorqueCompontent.Z` for heave
- `ForceTorqueCompontent.K` for roll
- `ForceTorqueCompontent.M` for pitch
- `ForceTorqueCompontent.N` for yaw


In [4]:
# List all the degrees of freedom that we want to control
dofs = [
    skadipy.allocator._base.ForceTorqueComponent.X,
    skadipy.allocator._base.ForceTorqueComponent.Y,
    skadipy.allocator._base.ForceTorqueComponent.N
]

Next, we create the allocator object using the thrusters and the degrees of freedom we want to control.

In [5]:
# Create the allocator object
allocator = skadirf.MinimumMagnitudeAndAzimuth(
    actuators=actuators,
    force_torque_components=dofs,
    gamma=0.5,
    mu=1.5,
    rho=1,
    time_step=0.2,
    control_barrier_function=skadipy.safety.ControlBarrierFunctionType.ABSOLUTE
)

To allow change in vehicle configuration, allocation matrix can be updated using `update_allocation_matrix` method.

Finally, we can use the allocator to allocate the forces and torques to the thrusters.

In [6]:
# Compute or update the configuration matrix
allocator.compute_configuration_matrix()

Regardless of the DOF we want to control, the allocator will always return forces and torques in the body frame.
If we want to control the vehicle in the NED frame, we need to rotate the forces and torques to the NED frame.

The $\tau_{\text{cmd}}$ **must** be in the form of

$$
\tau_{\text{cmd}} = \begin{bmatrix} F_x & F_y & F_z & M_x & M_y & M_z \end{bmatrix}
$$

where $F_x, F_y, F_z$ are the forces in the body frame and $M_x, M_y, M_z$ are the torques in the body frame.

In [7]:
tau_cmd = np.zeros((6, 1))

To get commanded forces to the thrusters, we can use the actuator objects.

In [8]:
from matplotlib.widgets import Slider as mplSlider, Button as mplButton

# Adjust the layout to include sliders and button in the figure
plt.ion()
fig, ax = plt.subplots(figsize=(10, 10))
plt.subplots_adjust(bottom=0.25)
# Create axes for sliders and button
axcolor = 'lightgoldenrodyellow'

ax_F_x = plt.axes([0.15, 0.15, 0.65, 0.03], facecolor=axcolor)
ax_F_y = plt.axes([0.15, 0.10, 0.65, 0.03], facecolor=axcolor)
ax_M_z = plt.axes([0.15, 0.05, 0.65, 0.03], facecolor=axcolor)
ax_button = plt.axes([0.8, 0.0, 0.1, 0.04])


# Create sliders and button
slider_F_x = mplSlider(ax_F_x, 'F_x', -10.0, 10.0, valinit=1.0)
slider_F_y = mplSlider(ax_F_y, 'F_y', -10.0, 10.0, valinit=0.0)
slider_M_z = mplSlider(ax_M_z, 'M_z', -10.0, 10.0, valinit=0.0)
button_allocate = mplButton(ax_button, 'Allocate')

# Function to update tau_cmd and allocate forces
def allocate_forces(event):
    global tau_cmd
    tau_cmd[0, 0] = slider_F_x.val
    tau_cmd[1, 0] = slider_F_y.val
    tau_cmd[5, 0] = slider_M_z.val
    allocator.allocate(tau=tau_cmd)

    positions = [actuator.position for actuator in actuators]
    forces = [actuator.force for actuator in actuators]
    # Extract X and Y components of positions and forces
    x_positions = [pos.x for pos in positions]
    y_positions = [pos.y for pos in positions]
    x_forces = [force[0] for force in forces]
    y_forces = [force[1] for force in forces]
    # Normalize the forces for better visualization
    scale_factor = 0.1
    x_forces = [force * scale_factor for force in x_forces]
    y_forces = [force * scale_factor for force in y_forces]

    # Create the plot with predefined limits
    ax.cla()
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-0.5, 0.5)
    ax.grid()
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    ax.set_title('Thrust Directions and Magnitudes in X-Y Plane')

    ax.quiver(x_positions, y_positions, x_forces, y_forces, angles='xy', scale_units='xy', scale=1, color='r')
    ax.scatter(x_positions, y_positions, color='b')
    fig.canvas.draw_idle()

# Attach the function to the button
button_allocate.on_clicked(allocate_forces)

0